In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import dtypes
from solarimage.classifier.nn.dataset import DataSet
from solarimage.classifier.nn.model.regression import log_linear
from solarimage.classifier.nn.optimizer.objective import genradvers
from solarimage.classifier.nn.optimizer.stepper import adam_optimizer

In [3]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [23]:
mnist = input_data.read_data_sets("example/MNIST_data/", one_hot=True)
train_dataset = DataSet(mnist.train.images, mnist.train.labels,reshape=False,dtype=dtypes.uint8)

Extracting example/MNIST_data/train-images-idx3-ubyte.gz


Extracting example/MNIST_data/train-labels-idx1-ubyte.gz
Extracting example/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting example/MNIST_data/t10k-labels-idx1-ubyte.gz


In [16]:
mb_size = 32
x_dim = 784
z_dim = 100
h_dim = 128
lr = 1e-3
d_steps = 3

In [17]:
# Discriminator Net
X = tf.placeholder(tf.float32, shape=[None, X_dim], name='X')

D_W1 = tf.Variable(xavier_init([x_dim, h_dim]), name='D_W1')
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]), name='D_b1')
D_W2 = tf.Variable(xavier_init([h_dim, 1]), name='D_W2')
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]

# Generator Net
Z = tf.placeholder(tf.float32, shape=[None, z_dim], name='Z')

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]), name='G_b1')
G_W2 = tf.Variable(xavier_init([h_dim, x_dim]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[x_dim]), name='G_b2')

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [18]:
G_sample, _ = log_linear(Z, G_W1, G_b1, G_W2, G_b2)
D_real, D_logit_real = log_linear(X, D_W1, D_b1, D_W2, D_b2)
D_fake, D_logit_fake = log_linear(G_sample, D_W1, D_b1, D_W2, D_b2)
D_loss, G_loss = genradvers(D_real, D_fake)

In [19]:
# Only update D(X)'s parameters, so var_list = theta_D
D_solver = adam_optimizer(D_loss, theta_D)
# Only update G(X)'s parameters, so var_list = theta_G
G_solver = adam_optimizer(G_loss, theta_G)

In [27]:
def sample_Z(m, n):
    '''Uniform prior for G(Z)'''
    return np.random.uniform(-1., 1., size=[m, n])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(1000000):
        #  train process
        x_mb, batch_ys = train_dataset.next_batch(mb_size, shuffle=False)
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: x_mb, Z: sample_Z(mb_size, z_dim)})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, z_dim)})